In [1]:
import jax
import jax.numpy as jnp
import numpy as np
import netket as nk
import flax.linen as nn

In [6]:
J = [1, 0.2]; L = 8; edge_colors = []
for i in range(L):
    edge_colors.append([i, (i+1)%L, 1])
    edge_colors.append([i, (i+2)%L, 2])
g = nk.graph.Graph(edges=edge_colors)
sigmaz = [[1, 0], [0, -1]];  mszsz = (np.kron(sigmaz, sigmaz))
exchange = np.asarray([[0, 0, 0, 0], [0, 0, 2, 0], [0, 2, 0, 0], [0, 0, 0, 0]])
bond_operator = [
        (J[0] * mszsz).tolist(),    (J[1] * mszsz).tolist(),
        (-J[0] * exchange).tolist(),(J[1] * exchange).tolist(),
        ]
bond_color = [1, 2, 1, 2];   sigmaz = [[1, 0], [0, -1]]
mszsz = (np.kron(sigmaz, sigmaz));
exchange = np.asarray([[0, 0, 0, 0], [0, 0, 2, 0], [0, 2, 0, 0], [0, 0, 0, 0]])
bond_operator = [
        (J[0] * mszsz).tolist(),    (J[1] * mszsz).tolist(),
        (-J[0] * exchange).tolist(),(J[1] * exchange).tolist(),
    ]

bond_color = [1, 2, 1, 2]

In [7]:
hi = nk.hilbert.Spin(s=0.5, total_sz=0.0, N=g.n_nodes)
op = nk.operator.GraphOperator(hi, graph=g, bond_ops=bond_operator, bond_ops_colors=bond_color)

In [8]:
# Definir o modelo RBM para representar o estado do sistema
alpha = 2
ma = nk.models.RBM(alpha=alpha)

In [13]:
# Configurar o otimizador para treinamento
op = nk.optimizer.Sgd(learning_rate=0.01)

# Configurar o VMC (Variational Monte Carlo) para treinamento
vmc = nk.VMC(hamiltonian=hi, optimizer=op, variational_state=ma, n_samples=1000)

# Configurar o GS (Ground State) para cálculo do estado fundamental
gs = nk.exact.GroundState(hamiltonian=hi)

AttributeError: "RBM" object has no attribute "hilbert". If "hilbert" is defined in '.setup()', remember these fields are only accessible from inside 'init' or 'apply'.

In [4]:
# Definir uma função de observável personalizada para gravar o bias e o kernel
def custom_observable(vstate):
    return {
        "bias": vstate.variables["params"]["bias"],
        "kernel": vstate.variables["params"]["kernel"]
    }

# Adicionar a função de observável personalizada ao VMC
vmc.add_observable(custom_observable, name="CustomObservables")

# Executar o VMC para treinamento e gravação dos observáveis no log
it = 100
gs.run(out='test', n_iter=it, obs={"CustomObservables": custom_observable(ma)})

NameError: name 'vmc' is not defined